### Trabalho de Programação

# Análise de dados de manchas solares 

#### Janaina Fernandes

### Proposta inicial:

Análise de dados de manchas solares no tempo e possível predição de ciclos futuros utilizando técnicas de Machine Learning. 

Utilizar autoencoders de CNN para melhorar o modelo teórico estabelecido.

### Dificuldades encontradas:


Dados e código do modelo teórico não disponíveis, análise teórica estatística, falta de familiaridade com a manipulação de séries temporais, dificuldade em encontrar códigos com abordagem prática devido às aplicações de séries temporais no mercado financeiro, necessidade de [feature design](http://www.cs.ucr.edu/~eamonn/meaningless.pdf) para a manipulação desse tipo de dado.

### Novo objetivo:

Análise exploratória dos dados de manchas solares, visualização, análise estatística e aplicação de métodos "clássicos": livro com abordagem prática para análise de séries temporais.

# Practical Time Series Analysis

### Master Time Series Data Processing, Visualization and Modeling using Python

Dr. Avishek Pal, Dr. PKS Prakash



*Notas*

livro de set/2017 - Atual

escrito por dois cientistas de dados

Modelos estatísticos + machine learning

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
%matplotlib inline

## Capítulo 1: Introdução a Séries Temporais

### Diferentes tipos de dados

-  Dados de corte transversal (cross-section)
-  Série temporal
-  Dados em painel (panel data)


*Notas*

dados de corte transversal: dados como o pib de uma lista de países no ano passado, dados descritivos da população brasileira, votação nas últimas eleições.

série temporal: dados de manchas solares, bolsa de valores, brl - usd

dados em painel: acompanhar os dados de corte transversal ao longo de um período de tempo, evolução dos indicadores de educação no brasil em um determinado período, gastos militares de usa, china, uk e india em % do pib.

Vamos analisar somente time series a fundo já aplicando qdo possível os dados de manchas solares

### Estruturas internas de séries temporais

-  Tendência
-  Sazonalidade (seasonality)
-  Movimentos cíclicos
-  Variações inesperadas

Baseado nesse modelo uma série temporal pode ser expressa por:

$$x_t = f_t+s_t+c_t+e_t$$

*notas*

Tendencia: pode ser em forma de reta, segunda ordem, exponencial

Sazonalidade: repetições periódicas em uma serie temporal. ao retirar a tendencia é possível observar

Movimentos cíclicos: ocorrem menos frequentemente que as flutuações sazonais, pode não ter um período fixo, assim como pra identificar tendencia, podem ser necessarios dados mto antigos.

Variações inesperadas: pode ser devido a falta de informação sobre variaveis que explicam esse fenomeno ou pela presença de random noise

*Notas*

O objetivo de uma série temporal é decompor nessas características acima e desenvolver modelos matemáticos pra cada uma. Esse modelo é utilizado para avaliar causas dos dados observados e predizer pontos futuros

### Modelos para análise de série temporal

-  Modelos de média zero
-  Random walk
-  Modelos de tendência
-  Modelos de sazonalidade


#### Modelos de média zero

Observações deste modelo são assumidas como sendo **independentes e identicamente distribuídas (iid)** e representam o random noise em torno da média fixa (subtraída da série temporal como termo constante).


#### Random walk

Dado como a soma de n iids. Pode ser reduzido ao modelo de média zero tomando as diferenças entre dois índices consecutivos:

$$ S_t-S_{t-1} = x_t$$

$x_t$ é iid com média zéro e variância constante.

#### Modelos de tendência

A escolha do modelo de tendência $\mu(t)$ é fundamental para a modelagem correta da série tempora. A análise exploratória dos dados normalmente dá dicas se devemos aplicar o modelo linear ou não linear em $t$.

$$\mu(t)=\omega t +b$$
$$\mu(t)=\omega_1 t+\omega_2 t^2 +b$$
$$\mu(t)=\omega_0 t^p +b$$

Exemplos de modelos lineares e não lineares de tendência.

#### Modelos de sazonalidade

Podem ser expressados pela soma ponderada de senos com peridiocidade conhecida:

$$s_t = \sum \frac{L}{k-1}(w_k\cos{\alpha t}+v_k\sin{\alpha t})+b$$

Também conhecido como modelo de regressão harmônica.

### Passo a passo para análise de uma série temporal

-  Visualização de dados com diferentes granularidades de tempo para revelar tendência e flutuações sazonais.
-  Identificação da tendência e plot do resíduo para identificar sazonalidade ou erro irredutível.
-  Modelo de regressão harmônica para capturar a sazonalidade.
-  Plotar o erro residual restante e identificar se é realmente erro irredutível.

### Autocorrelação e Autocorrelação parcial

Depois de aplicar as tranformações anteriores ficamos com o que é chamado de **série temporal estacionária**, caracterizada por uma média constante $E(x_t)$ e correlação dependente somente da diferença de tempo $h$ entre as duas observações e são independentes de seu valor $$E(x_t,x_{t+h})=g(h)$$ 

*Notas*

Quando normalizadas entre -1 e 1, chamamos essa relação de autocorrelação, isso garante que a série é independente do tempo e tem características que podem ser úteis na predição futura da série temporal.

correlação positiva sobe junto e negativa observáveis são inversamente proporcionais. Quanto mais próximo de zero menor a correlação e mais difícil de prever

#### Função de autocorrelação (ACF)

Plot ACF em forma de mapa para vários lags.

Plot ACF com o intervalo de confiança de 95%. Autocorrelações fora do intervalo são estatisticamente relevante, dentro do intervalo são devidas ao ruído.



#### Autocorrelação Parcial

**Modelos autoregressivos (AR)** capturam a dependencia como regressão linear entre $x_{t+h}$ e $x_t$ mas da forma calculada anteriormente a influência dos valores intermediários  $x_t, x_{t+1},...,x_{t+h-1}$ também são consideradas.

A autocorrelação parcial resolve esse problema definindo a correlação entre $x_{t+h}$ e $x_t$ desconsiderando os lags $t+1$ até $t+h-1$.

Ajuda na identificação da ordem $h$ de um modelo $AR(h)$.



*Notas*

-  Plot_pacf, pontos acima do intervalo de confiança maiores que 1 (1 sempre está acima) nos dão a ordem: 1 ponto ordem 1, 2 pontos ordem 2...

In [2]:
# Pré-tratamento dos dados
df = pd.read_csv('SN_m_tot_V2.0.csv', sep=';', header=None)
df.columns = ['year', 'month', 'FracDate', 'Sunspots', 'Std', 'Observations', 'Verified']
df = df[['year', 'month', 'Sunspots']]

# #df['Date'] = pd.to_datetime(df["Date"])
# #df_idx = df.set_index(["Date"], drop=True)
df['Date'] = df.apply(lambda row: datetime(
                              int(row['year']), int(row['month']),1), axis=1)
df['Date'] = pd.to_datetime(df["Date"])
df_idx = df.set_index(["Date"], drop=True)
df_idx.head(5)


,year,month,Sunspots
Date,,,
1749-01-01,1749,1,96.7
1749-02-01,1749,2,104.3
1749-03-01,1749,3,116.7
1749-04-01,1749,4,92.8
1749-05-01,1749,5,141.7


## Capítulo 2: Entendendo os dados de uma série temporal

<img src="fluxograma.png">


### Processos estacionários

Um **processo estacionário** requer que média, variância e autocorrelação sejam invariantes com respeito ao tempo da observação.

No entanto, a maioria dos processos reais em séries temporais não são estacionários. Uma etapa crucial na análise de séries temporais é verificar se uma série é estacionária e métodos para tornar processos não estácionários em estacionários, removendo tendência e sazonalidade.

### Diferenciação

Podendo ser de primeira ordem ou de segunda ordem para eliminação da tendência:

$$\dot{x}_t = (x_t-x_{t-1})$$
$$\ddot{x}_t = (\dot{x}_t-\dot{x}_{t-1})= (x_t-x_{t-1})-(x_{t-1}-x_{t-2})$$

Se a série temporal tem sazonalidade com período conhecido $m$ a difereciação retira este efeito da série:

$$\dot{x}_t = x_t-x_{t-m} = \epsilon_t$$

### Teste de Dickey-Fuller aumentado

Teste com $H_0$ sendo um processo não estacionário e a hipótese alternativa um processo estácionário.


### Decomposição de uma série temporal

-  Modelo aditivo 
$$x_t=F_t+S_t+\epsilon_t$$
-  Modelo multiplicativo
$$x_t=F_t+S_t+\epsilon_t$$
$$\log{x_t}=\log{F_t}+\log{S_t}+\log{\epsilon_t}$$


### Médias móveis

**Médias móveis (MA)** estimam a tendência média calculada pela média no período de $t\pm k$ onde $k$ é o alcance da MA:

$$F_t=\frac{x_{t-k}+x_{t-k+1}+...+x_t+...+x_{t+k}}{2k+1}$$

Usualmente nos referimos ao número total de observações $m=2k+1$ como a MA de ordem $m$.

As médias móveis podem ser usadas para a suavização dos gráficos, foram descritas de forma simétrica a $x_t$ para o caso de $m$ ímpar mas podemos considerar ainda médias assimétricas $m=2k$.

A assimetria de uma MA de ordem par pode ser eliminada tomando a segunda MA de ordem par. Como por exemplo no caso de $m=2$:

$$F_t^{(2)}=\frac{x_{t-1}+x_{t}}{2}$$

Aplicando uma segunda média móvel de segunda ordem à série $F_t^{(2)}$ produziremos uma média movel final simétrica:

$$2\times F_t^{(2)}=\frac{F_t^{(2)}+F_{t+1}^{(2)}}{2}=\frac{1}{2}\times \frac{x_{t-1}+x_t}{2}+\frac{x_{t}+x_{t+1}}{2}$$

$$=\frac{1}{4}x_{t-1}+\frac{1}{2}x_t+\frac{1}{4}x_{t+1}$$


Em geral, podemos criar as médias móveis $n\times F_t^{(m)}$ tomando primeiro a média de ordem $m$ seguida da média de ordem $n$. Para produzir uma MA simétrica $n$ e $m$ devem ser os dois pares ou os dois ímpares.

*Notas*

Uma característica importante é que podemos chamar de média móvel ponderada, quanto mais próximo de $x_t$ maior o peso dado à variável, o que gera um conjunto de dados com curva mais suave.

### Ajuste sazonal usando médias móveis

Podemos utilizar as séries $n\times F_t^{(m)}$ descritas anteriormente como médias ponderadas que podem ser aplicadas para suavizar dados com sazonalidade para gerar estimativas de tendência. Por exemplo para dados trimestrais podemos aplicar $2\times F_t^{(4)}$ (4 é o número de trimestres em um ano). A MA é tratada como a tendência dos dados e o resíduo avaliado pelo teste ACF.


### Decomposição de série temporal

#### Por médias móveis

Pela abordagem descrita temos que:

$$\epsilon_t=x_t-F_t-S_t$$
No modelo aditivo.

$$\epsilon_t=\frac{x_t}{F_t\times S_t}$$
No modelo multiplicativo.

#### Usando statsmodels.tsa

Pacote que já tem esses modelos e precisamos apenas dar os parâmetros da decomposição.

## Capítulo 3: Métodos baseados em suavização exponencial

O processo de predição baseado na suavização exponencial pode ser resumido como filtros que tomam inputs e separam os componentes tendência e ruído como na figura:

FIGURA CAP 3



### Suavização exponencial de primeira ordem

É plicada em dados de variância constante e sem sazonalidade. Essa abordagem é recomendada para predições de curto prazo.

A fórmula geral é dada por:
$$x_t=\alpha x_t+(1-\alpha)x_{t-1}$$

Aqui, o modelo é a média ponderada entre o valor atual e o valor anterior com um fator de suavização $\alpha$ que nos dá o quão rápido ''esqueceremos'' a última observação. Quanto menor o $\alpha$ maior é a influência dos valores anteriores e mais suave é a série.

A recursividade exponencial está na multiplicação a cada $t$ por $(1-\alpha)$.

A acurácia do modelo pode ser avaliada por funções como o **erro absoluto médio (MAS)** e o **erro quadrado médio (MSE)**

$$MSE =\frac{1}{N} \sum ^N_{i=1}(x_t-F_t)^2$$

$$MAD=\frac{1}{N} \sum ^N_{i=1}|x_t-F_t|$$

### Suavização exponencial de segunda ordem

Aplicaremos a suavização anterior à tendência assumindo que a direção das mudanças da série temporal dependem da diferença dos pesos anteriores. Como resuldados tempos as seguintes funções com nível $\ell$ e tendência $b$:

$$\ell_t = \alpha x_t + (1-\alpha)(\ell_{t-1} + b_{t-1})$$

$$b_t = \beta(\ell_t - \ell_{t-1}) + (1-\beta)b_{t-1}$$

$$x_{t+1} = \ell_t + b_t$$

$\beta$ é o coeficiente de suavização da segunda exponencial. A predição é a soma do modelo de valores de $\ell$ e $b$.

Agora temos dois parâmetros $\alpha$ e $\beta$. O primeiro suaviza a curva em torno da tendência e o segundo suaviza a própria tendência.

## Capítulo 4: Modelos Autorregressivos

### Modelos autorregressivos
**Modelos autorregressivos (AR)** são modelos estatísticos muitos bons em predição de séries temporais em que os valores seguintes dependem dos valores anteriores. Assim, modelos AR são muito utilizados na predição de séries temporais.

Os modelos autorregressivos são definidos como AR$(p)$ onde $p$ é a ordem do comporente AR.

O modelo AR de primeira ordem, AR(1):
$$x_t=\phi x_{t-1}+\in_t$$

AR(2):

$$x_t=\phi_1 x_{t-1}+\phi_2 x_{t-2}+\in_t$$

O modelo AR de p-ésima ordem, AR$(p)$:
$$x_t=\phi_1 x_{t-1}+\phi_2 x_{t-2}+...+\phi_p x_{t-p}+\in_t$$

$\phi$ é o coeficiente do modelo, $\in_t\approx N(0,\sigma^2)$ é o erro no tempo $t$.

### Modelos de Médias Móveis:

Similarmente ao modelo AR o **modelo de médias móvias (MA)** tem o mesmo objetivo e ordem $q$, porém nestes modelos a suposição é que o erro atual depende do erro anterior.

### Modelo ARMA 

A combinação dos modelos anteriores AR$(p)$ e MA$(q)$ forma o modelo ARMA$(p,q)$:

$$ x_t = \alpha + \phi_1 x_{t-1} + \phi_2 x_{t-2} +...+ \phi_p x_{p-q}-\theta_1\epsilon_{t-1}-\theta_2\epsilon_{t-2}-...-\theta_q\epsilon_{t-q}+\epsilon_t $$

Algumas regras gerais para selecionar o $p$ e $q$ do modelo ARMA a ser utilizado:

-  Autocorrelação decrescente exponencialmente e PACF com correlação significativa usando lag 1, então usamos o parâmetro $p$
-  Autocorrelação formando um padrão seno e PACF com correlação significativa com lags 1 e 2, usamos $p=2$
-  Autocorrelação é significativa e PACF cai exponencialmente, usar parâmetro $q$
-  Autocorrelação é significativa e PACF tem padrão seno, usar parâmetro $q$



### Modelo ARIMA

É uma generalização do modelo ARMA que inclui componentes integradas.

Uma extensão importante do modelo ARIMA é a inclusão da sazonalidade no modelo. Ele possui então seis parâmetros ARIMA$(p,d,q)(P,D,Q)_m$ onde P, D e Q representam a parte sazonal do modelo autorregressivo. O $m$ é o número de períodos na série.


In [1]:
print('olar')

olar


## Capítulo 5: Deep Learning para predição de série temporal

